## In what ways has social media contributed to the growth and visibility of the Fridays for Future climate movement?


In [ ]:
from googleapiclient.discovery import build
from collections import OrderedDict
api_key = ""
youtube = build("youtube", "v3", developerKey= api_key)

# Setting parameters for video search
all_videos = []
next_page_token = None
max_pages = 3
current_page = 0

while current_page < max_pages:
    request = youtube.search().list(
        q="Fridays for Future",
        maxResults=50,
        part="snippet",
        order="relevance",
        regionCode="DE",
        type="video",
        publishedAfter="2018-01-01T00:00:00Z",
        publishedBefore="2021-01-01T00:00:00Z",
        pageToken=next_page_token
    )
    response = request.execute()
    all_videos.extend(response["items"])

    next_page_token = response.get("nextPageToken")
    current_page += 1
    
#List of video ids
ids = []
for item in all_videos:
    ids.append(item["id"]["videoId"])

#handle duplicates 
unique_ids= list(set(ids))
print(unique_ids)

titles = []
for item in all_videos:
    titles.append(item["snippet"]["title"])
    

#getting comments for each video
all_comments = [] 
for videoid in unique_ids:
    video_comments = []
    try: 
      request = youtube.commentThreads().list(
        part="snippet",
        videoId= videoid,
        maxResults= 10,
        testFormat="plainText",
        order= "relevance"        
    )
      response = request.execute()
      for c in response.get("items", []):
         video_comments.append(c["snippet"]["topLevelComment"]["snippet"]["textDisplay"])
          
    #Comments disabled
    except Exception as e:
        continue
    all_comments.append(video_comments)

descriptions =[]
for video_id in unique_ids:
    try:
        request = youtube.videos().list(
            part="snippet",
            id=video_id
        )
        response = request.execute()
        for item in response.get("items", []):
            description = item["snippet"].get("description", "")
            descriptions.append(description)

    except Exception as e:
         continue
        
final_data = all_comments + titles + descriptions
print(final_data)



    

In [ ]:
video_data = []

for item in unique_ids:
    request = youtube.videos().list(
        part="snippet, statistics",
        id=item
    )
    response = request.execute()
    
    for video in response.get("items", []):
        title = video["snippet"].get("title", "")
        stats = video.get("statistics", {})
        views = int(stats.get("viewCount", 0))
        likes = int(stats.get("likeCount", 0))
        comments = int(stats.get("commentCount", 0))
        upload_time = video["snippet"].get("publishedAt", "")
        
        video_data.append({
            "title": title,
            "views": views,
            "likes": likes,
            "comments": comments,
            "upload time": upload_time
            
        })

#Sort by view count
sorted_videos = sorted(video_data, key=lambda x: x['views'], reverse=True)

for item in sorted_videos:
    print(item)

In [ ]:
import pandas as pd
#Counts how many videos were uploaded on each day
df["upload time"] = pd.to_datetime(df["upload time"])
df["year_month"] = df["upload time"].dt.to_period("M")
video_counts = df["year_month"].value_counts().sort_index()

print(video_counts)


In [ ]:
from collections import Counter
import pandas as pd

keywords_list = []
for item in final_data:
    # Handles comments being lists
    text = str(item)
    kw_single = model.extract_keywords(text, keyphrase_ngram_range=(1,1))
    kw_double = model.extract_keywords(text, keyphrase_ngram_range=(1,2))
    keywords_list.extend(kw_single + kw_double)

# Extract keywords
ignore_words = ["fridays", "fridays future", "future", "fridaysforfuture", "auf", "thunberg","ich", "nicht", "greta", "klima", "und"]
flattened_keywords = [kw for kw, score in keywords_list if kw.lower() not in ignore_words]


# Count frequencies
keywords_df = pd.DataFrame(Counter(flattened_keywords).items(), columns=["words", "numbers"])
frequent_words = keywords_df.loc[keywords_df['numbers'] > 1].sort_values(by='numbers', ascending=False)
print(frequent_words.head(10))


